In [87]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [88]:
# G마켓 슈퍼딜 사이트
url = 'http://corners.gmarket.co.kr/SuperDeals/' # 추천 카테고리

# 웹 서버 요청
resp = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(resp.text, 'html.parser')

In [89]:
item_list_tag = soup.find('ul', attrs={'class':'item_list'})
item_list_tags = item_list_tag.find_all('li')

title_list = []
sale_list = []
price_list = []
buy_list = []

for li_tag in item_list_tags:
    title_text = li_tag.find('span', attrs = {'class': 'title'}).get_text()
    title_list.append(title_text)
    
    sale_text = li_tag.find('em', attrs = {'class' : 'sale'})
    if sale_text is None:
        sale_list.append('0')
    else:
        sale_list.append(sale_text.get_text().strip('% 할인'))

    price_text = li_tag.find('span', attrs = {'class': 'price'}).find('strong').get_text().strip('원') # rent-per 부분 처리를 어떻게 할까?
    price_list.append(price_text)

    buy_text = li_tag.find('span', attrs = {'class':'buy'}).find('strong').get_text()
    buy_list.append(buy_text)


print(title_list)
print(sale_list)
print(price_list)
print(buy_list)

['(15%+10%)큐어 하이드라 수딩토너&에멀젼 130ml', '부광 얇은헤드 칫솔 초극세모 12입 X 3세트', '[15%쿠폰] 대용량 보스턴백 캐리어 결합백 빅사이즈 백 숄더백', '정샘물 에센셜 스킨 누더 쿠션(리필 포함)', '[15%쿠폰] 시즌오프 티셔츠/데님자켓/데님팬츠 외', '순수식품 종합비타민 미네랄 20종 12개월분+사은품', '전통식품인증 국내산 파김치 1KG', '새치커버력 탈모개선 소소해 데일리 블랙샴푸 300ml', '1+1 특가 남녀공용 봄반팔 골라담기', 'SK매직정수기 상품권30만+10%할인', 'LG정수기 상품권30만+결합할인 3천원!', '탑텐 본사직영 베이직/팬츠 외 간절기 BEST 특가전', '[쿠폰가 41,900원] 이마트인기 이맛쌀 20kg', '[15%] 종근당건강 락토핏 골드 보장균수 2배 50포 3통(150포)', '서울멸균우유 1000ml x 10입 (1박스)', '나이키/뉴발란스 반팔티 긴바지 모음', '[15%] 프리미어 70매 캡 20팩#26,990원', '[당일배송] 동서식품 맥심 모카골드 커피믹스 230T', '본사직영/폴햄 봄 신상 및 클리어런스 맨투맨/티셔츠/조거팬츠外', '(12400원)피죤 3100mlx2개+2300mlx3개+300ml증정', '[10%] 순수 3겹 천연펄프 화장지 30m 30롤 3팩', '블루독/블루독베이비 SS여름오픈 기념 인하 ~60%OFF+무배', '쿠폰가 38900원)셀렉스 프로틴음료 오리지널 125ml 48팩+증정', '[15%]뮬라웨어 SS레깅스 애슬레저', '(15%) 4중 브러쉬 포크 타투 아이브로우 틴트 3개', '[크림증정] 청귤 비타C 잡티케어 기획세트', '수퍼슈프림 리치골드 L 세트/방문포장 전용', '[15+10%쿠폰] 매일 입게 될 봄 스타일 제안', '[15%+5%]플레이보이 남자 드로즈 10매입 세트', '잘풀리는집 순수PURE천연펄프 25m 30롤 3팩', '(선물하기 특가) KFC 더블징거블랙라벨세트', '[15+10%쿠폰] 크로커다일外 형지패

In [90]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가
dict_data = {
    'title' : title_list,
    'sale' : sale_list,
    'price' : price_list,
    'buy' : buy_list
}

# 판다스 데이터프레임으로 변환
df_recom = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_recom

,title,sale,price,buy
0,(15%+10%)큐어 하이드라 수딩토너&에멀젼 130ml,50,"20,900","2,353"
1,부광 얇은헤드 칫솔 초극세모 12입 X 3세트,30,"17,000",1만+
2,[15%쿠폰] 대용량 보스턴백 캐리어 결합백 빅사이즈 백 숄더백,70,"9,900","1,271"
3,정샘물 에센셜 스킨 누더 쿠션(리필 포함),15,"35,700",863
4,[15%쿠폰] 시즌오프 티셔츠/데님자켓/데님팬츠 외,69,"32,000","1,253"
...,...,...,...,...
292,리큅 미생물 음식물처리기 LFC-MO20 2kg,0,"579,000",1
293,차량용 공기청정기 AG25,0,"149,000",48
294,인터넷+TV 결합 47만원 증정+상담시 커피쿠폰!,0,"34,100",13
295,제주도 실시간 편도 항공권,0,"76,000",NEW


## 카테고리별로 크롤링

In [91]:
def category_url(num,cate):
    url = 'http://corners.gmarket.co.kr/SuperDeals/?DispType=1&GroupNo=400000{}&SubNo=0&CateIndex=0'.format(num)
    
    # 웹 서버 요청
    resp = requests.get(url)

    # HTML 파싱
    soup = BeautifulSoup(resp.text, 'html.parser')

    item_list_tag = soup.find('ul', attrs={'class':'item_list'})
    item_list_tags = item_list_tag.find_all('li')

    title_list = []

    for li_tag in item_list_tags:
        title_text = li_tag.find('span', attrs = {'class': 'title'}).get_text()
        title_list.append(title_text)

    # 딕셔너리 형식으로 항목별 리스트를 원소로 추가
    dict_data = {
        'title' : title_list
    }

    # 판다스 데이터프레임으로 변환
    df_data = pd.DataFrame(dict_data)
    df_data['category'] = cate

    # 변환 결과를 확인
    return df_data

In [92]:
category = {
    '브랜드패션'  : 135,
    '트랜드패션'  : 136,
    '뷰티·잡화'  : 137,
    '유아동'      : 138,
    '식품'        : 139,
    '생필품'      : 140,
    '가구·침구'  : 141,
    '생활·건강'  : 143,
    '스포츠·레저': 146,
    '가전·컴퓨터': 148,
    '디지털'      : 149, 
    '문구·도서'  : 151,
    '여행'        : 152,
    'e쿠폰'       : 153,
    '백화점'      : 164
}

In [95]:
df_cate = pd.DataFrame()
for key in category:
    cate = key
    value = category.get(cate)
    df_cate = pd.concat([df_cate, category_url(value, cate)], ignore_index = True)

df_cate

,title,category
0,[15%쿠폰] 시즌오프 티셔츠/데님자켓/데님팬츠 외,브랜드패션
1,탑텐 본사직영 베이직/팬츠 외 간절기 BEST 특가전,브랜드패션
2,본사직영/폴햄 봄 신상 및 클리어런스 맨투맨/티셔츠/조거팬츠外,브랜드패션
3,[15+10%쿠폰] 매일 입게 될 봄 스타일 제안,브랜드패션
4,[15+10%쿠폰] 크로커다일外 형지패션 여성의류 특가전,브랜드패션
...,...,...
1767,클라이드 공용 스누피 스타디움 자켓 외,백화점
1768,아디다스골프 남성 퍼포먼스 클래식,백화점
1769,블랙야크키즈 BKO카메론맨투맨세트,백화점
1770,[레노마_셔츠(남성)] 2023 레노마 스판 드레스셔츠 슬림/일반핏 특집전,백화점
